In [1]:
import pandas as pd
from collections import OrderedDict
from collections import Counter
import sys
import os
from glob import glob

In [2]:
paths_2016 = glob("../../../../../Documents/data/2016-12-to-2017-03/non-dod/status/*.txt")
paths_2016

['../../../../../Documents/data/2016-12-to-2017-03/non-dod/status\\Non-DoD FOIA 2017-04762 201612.txt',
 '../../../../../Documents/data/2016-12-to-2017-03/non-dod/status\\Non-DoD FOIA 2017-04762 201703.txt']

In [3]:
data_2016 = pd.concat(
    [pd.read_csv(
        path, 
        sep=";", 
        low_memory=False).assign(**{
        "Name": lambda x: x[["Last Name", "First Name"]].fillna("").apply(",".join, axis=1)}) 
     for path in paths_2016], 
axis=0)
data_2016.head()

,Last Name,First Name,File Date,Agency,SubAgency,State,Age Range,YSD Range,Education Level,Pay Plan,Grade,LOS Level,Occupation,PATCO,Adjusted Basic Pay,Supervisory Status,TOA,Work Schedule,NSFTP Indicator,Name
0,MAZZEI,JOHN,201612,VA-DEPARTMENT OF VETERANS AFFAIRS,VALA-VETERANS BENEFITS ADMINISTRATION,39-OHIO,50-54,20 - 24 years,13-BACHELOR'S DEGREE,GS-GENERAL SCHEDULE,05,Less than 1 year,0901,A,41433.00,8-ALL OTHER POSITIONS,15-CAREER-CONDITIONAL (COMPETITIVE SERVICE PEM...,F-FULL-TIME,1-NSFTP,"MAZZEI,JOHN"
1,NAME WITHHELD BY OPM,NAME WITHHELD BY OPM,201612,TR-DEPARTMENT OF THE TREASURY,TR93-INTERNAL REVENUE SERVICE,REDACTED,35-39,15 - 19 years,13-BACHELOR'S DEGREE,GS-GENERAL SCHEDULE,12,15 - 19 years,1169,A,80481.00,8-ALL OTHER POSITIONS,10-CAREER (COMPETITIVE SERVICE PERMANENT),F-FULL-TIME,1-NSFTP,"NAME WITHHELD BY OPM,NAME WITHHELD BY OPM"
2,NAME WITHHELD BY OPM,NAME WITHHELD BY OPM,201612,TR-DEPARTMENT OF THE TREASURY,TR93-INTERNAL REVENUE SERVICE,REDACTED,35-39,5 - 9 years,13-BACHELOR'S DEGREE,GS-GENERAL SCHEDULE,11,5 - 9 years,0512,P,79711.00,8-ALL OTHER POSITIONS,10-CAREER (COMPETITIVE SERVICE PERMANENT),F-FULL-TIME,1-NSFTP,"NAME WITHHELD BY OPM,NAME WITHHELD BY OPM"
3,NAME WITHHELD BY OPM,NAME WITHHELD BY OPM,201612,TR-DEPARTMENT OF THE TREASURY,TR93-INTERNAL REVENUE SERVICE,12-FLORIDA,45-49,10 - 14 years,16-POST-FIRST PROFESSIONAL,GS-GENERAL SCHEDULE,15,10 - 14 years,0905,P,129115.00,8-ALL OTHER POSITIONS,30-SCHEDULE A (EXCEPTED SERVICE PERMANENT),F-FULL-TIME,1-NSFTP,"NAME WITHHELD BY OPM,NAME WITHHELD BY OPM"
4,NAME WITHHELD BY OPM,NAME WITHHELD BY OPM,201612,TR-DEPARTMENT OF THE TREASURY,TR93-INTERNAL REVENUE SERVICE,REDACTED,35-39,15 - 19 years,13-BACHELOR'S DEGREE,GS-GENERAL SCHEDULE,12,10 - 14 years,0512,P,84557.00,8-ALL OTHER POSITIONS,10-CAREER (COMPETITIVE SERVICE PERMANENT),F-FULL-TIME,1-NSFTP,"NAME WITHHELD BY OPM,NAME WITHHELD BY OPM"


In [4]:
occupation_names = pd.read_csv("DTocc.txt").rename(columns={"OCCT": "Occupation Name"})
occupation_names.head()

,OCCTYP,OCCTYPT,OCCFAM,OCCFAMT,OCC,Occupation Name
0,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0006,0006-CORRECTIONAL INSTITUTION ADMINISTRATION
1,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0007,0007-CORRECTIONAL OFFICER
2,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0017,0017-EXPLOSIVES SAFETY
3,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0018,0018-SAFETY AND OCCUPATIONAL HEALTH MANAGEMENT
4,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0019,0019-SAFETY TECHNICIAN


In [5]:
occupations = Counter(data_2016["Occupation"])
occupation_counts = pd.merge(
    occupation_names, 
    pd.DataFrame.from_dict(occupations, orient="index").reset_index(), 
    left_on="OCC", 
    right_on="index")[["Occupation Name", 0]]\
    .rename(columns={"Occupation Name": "Occupation", 0: "Count"})\
    .sort_values(["Count"], ascending=False)
occupation_counts.to_csv("occupation_counts.csv", index=False)
occupation_counts.head()

,Occupation,Count
146,0610-NURSE,149530
62,0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM,126000
356,1802-COMPLIANCE INSPECTION AND SUPPORT,103961
404,2210-INFORMATION TECHNOLOGY MANAGEMENT,94381
79,0343-MANAGEMENT AND PROGRAM ANALYSIS,92121
